# Agents Can Code

"AI-generated code is full of bugs."

"You still need to understand everything it writes."

"It's just autocomplete."

Let's see.

## ChatGPT vs Coding Agents

| ChatGPT | Coding Agent |
|---------|--------------|
| You paste code | Agent finds the code |
| You describe the bug | Agent reads and understands |
| You copy the fix | Agent edits the file |
| You test it | Agent runs tests |
| You fix the next error | Agent iterates until done |

**Agents don't need babysitting.** They find context, try things, and iterate.

## The Tools

| Tool | Purpose |
|------|---------|
| `glob_files` | Find files by pattern |
| `grep_files` | Search for text in files |
| `read_file` | Read file contents |
| `edit_file` | Modify existing files |
| `write_file` | Create new files |
| `execute_bash` | Run commands |

In [23]:
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(Path.cwd() / ".env")

True

In [24]:
# Reset example file to its original buggy state (makes notebook idempotent)
BUGGY_CALCULATOR = '''\
"""A simple calculator module with a bug for the coding agent to fix."""


def divide(a, b):
    """Divide a by b."""
    return a * b  # BUG: should be a / b


def average(numbers):
    """Calculate the average of a list of numbers."""
    return sum(numbers) / len(numbers)
'''

def reset_example():
    """Reset the example file to its buggy state."""
    Path("examples/buggy_calculator.py").write_text(BUGGY_CALCULATOR)
    print("Reset examples/buggy_calculator.py to buggy state")

In [25]:
from agents import Agent, Runner as AgentRunner
from omniagents import Runner
from omniagents.builtin.tools import (
    read_file,
    write_file,
    edit_file,
    glob_files,
    grep_files,
    execute_bash,
)

In [26]:
reset_example()

Reset examples/buggy_calculator.py to buggy state


## Create the Coding Agent

In [18]:
INSTRUCTIONS = """
You are a skilled programmer. When asked to fix bugs or modify code:

1. **Locate** - Use glob_files or grep_files to find relevant files
2. **Understand** - Always read_file before editing. Never modify code you haven't seen.
3. **Edit** - Use edit_file to make targeted changes. Keep edits minimal and focused.
4. **Create** - Use write_file to create new files when needed.
5. **Verify** - Run tests or the code to confirm your fix works

Be concise in your explanations. Show what you changed and why.
"""

coder = Agent(
    name="Coding Agent",
    instructions=INSTRUCTIONS,
    tools=[read_file, write_file, edit_file, glob_files, grep_files, execute_bash],
    model="gpt-4.1",
)

## Example 1: Fix a Bug

```python
def divide(a, b):
    """Divide a by b."""
    return a * b  # BUG: should be a / b
```

Can the agent find and fix this?

In [19]:
runner = Runner.from_agent(coder)
runner.run_notebook(
    input="There's a bug in examples/buggy_calculator.py - the divide function returns wrong results. Fix it."
)

## Example 2: Add a Feature

Can the agent add a `safe_divide` function that handles division by zero?

In [8]:
runner = Runner.from_agent(coder)
runner.run_notebook(
    input="Add a 'safe_divide' function to examples/buggy_calculator.py that returns None instead of raising an error when dividing by zero."
)

## Example 3: Search and Fix

Can the agent find all `average` functions and make them handle empty lists?

In [9]:
runner = Runner.from_agent(coder)
runner.run_notebook(
    input="Search the examples/ folder for any function that calculates an average. Add a check to handle empty lists (return 0 instead of crashing)."
)

## Try It Yourself

In [ ]:
runner = Runner.from_agent(coder)
runner.run_notebook()

## The Key Principle

**Read before you write.**

| Step | Action |
|------|--------|
| 1 | `read_file` → see the code |
| 2 | Understand the bug |
| 3 | `edit_file` → fix it |
| 4 | `execute_bash` → verify |

The agent examines code before modifying it.

## Safety: Approval Before Action

The agent can modify files without asking. That's risky.

**SafeAgent** adds an approval layer:
- ✅ Auto-approve: `read_file`, `glob_files`, `grep_files` (read-only)
- ⚠️ Ask permission: `edit_file`, `write_file`, `execute_bash` (dangerous)

In [ ]:
# Same agent, but with safe=True
safe_runner = Runner.from_agent(
    coder,
    safe=True,  # Enable approval prompts for dangerous tools
    safe_tool_names=["read_file", "glob_files", "grep_files"],  # These don't need approval
)

Reset examples/buggy_calculator.py to buggy state


## SafeAgent Demo

You'll see approval prompts before any file modifications:

In [ ]:
# Reset the file so SafeAgent has a bug to fix
reset_example()

In [ ]:
safe_runner.run_notebook(
    input="There's a bug in examples/buggy_calculator.py - the divide function returns wrong results. Fix it."
)

## Example 4: Build an Entire Game

Fixing bugs is one thing. Can the agent build something **from scratch**?

Let's ask it to build a complete arcade game:
- Type A → Z as fast as you can
- Sound effects for success/failure
- High score leaderboard with initials
- Retro arcade styling
- Tested game logic

In [28]:
from omniagents.builtin.tools import apply_patch

GAME_DEV_INSTRUCTIONS = """
You are an expert game developer and software engineer. You build complete, 
polished applications autonomously.

## Your Approach
1. **Plan first** - Before writing any code, create a clear implementation plan
2. **Execute systematically** - Work through the plan step by step
3. **Test as you go** - Write tests for core logic and run them
4. **Iterate on failures** - If something doesn't work, debug and fix it

## Tools
- `glob_files` / `grep_files` - Find and search files
- `read_file` - Always read before modifying
- `write_file` - Create new files
- `apply_patch` - Modify existing files using unified diff format
- `execute_bash` - Run commands (install deps, run tests, etc.)

## Code Quality
- Write clean, well-structured TypeScript/JavaScript
- Include proper error handling
- Add comments for complex logic
- Follow modern React patterns

## Autonomous Operation
- Do NOT ask for confirmation between steps
- Make reasonable decisions and proceed
- Only stop if you encounter an unrecoverable error
"""

game_dev = Agent(
    name="Game Developer",
    instructions=GAME_DEV_INSTRUCTIONS,
    tools=[read_file, write_file, apply_patch, glob_files, grep_files, execute_bash],
    model="gpt-5.2",
)

In [37]:
import shutil

# Clear the fingerfrenzy directory to make the notebook idempotent
fingerfrenzy_path = Path("examples/fingerfrenzy")
if fingerfrenzy_path.exists():
    shutil.rmtree(fingerfrenzy_path)
    print("Cleared examples/fingerfrenzy/")
fingerfrenzy_path.mkdir(parents=True, exist_ok=True)
print("Ready to build Finger Frenzy!")

Cleared examples/fingerfrenzy/
Ready to build Finger Frenzy!


In [38]:
runner = Runner.from_agent(game_dev, max_turns=10000)
runner.run_notebook(
    input="""Build an arcade-style typing game called "Finger Frenzy" in examples/fingerfrenzy/

## Game Requirements
- The player must type A, B, C, D ... Z as fast as they can
- The primary visual should be the current letter
- Do not display all the letters on the screen, as the user knows the alphabet
- Timer starts when they hit A, ends when they hit Z
- Spacebar should trigger a reset
- Sound effects: success sound on correct key, error sound on wrong key, reset sound on reset
- When they hit Z, it should display their time and the leaderboard
- Leaderboard shows top 10 scores with initials and times
- If they achieve a high score (they beat out a score in the current top 10) then prompt for 3-letter initials to add to leaderboard
- Clear the textbox for top 10 entry is empty when displayed (it should not have 'Z')
- Everything should be keyboard driven and not require mouse interaction
- Spacebar while on the leaderboard should reset so they can play again
- Retro arcade visual style (think 80s arcade cabinet)
- This should be fun, engaging, and viral


## Technical Requirements  
- Use TypeScript and React
- Game logic should be well tested (unit tests, no browser/E2E tests needed)
- Install any dependencies needed
- The game should be runnable with npm run dev

## Approach
1. First, create a plan for the implementation
2. Then execute the plan autonomously without asking for confirmation
3. Run tests to verify the game logic works
4. Provide instructions for how to run the game when complete
"""
)

## What Just Happened?

From a single prompt, the agent:
- Created a project structure
- Installed dependencies
- Wrote game logic
- Built React components
- Added sound effects
- Styled it retro arcade
- Wrote and ran tests
- Delivered a working game

No copy-paste. No babysitting. No "fix this error."

## "Should I Be Worried?"

You might be thinking:

*"If AI can code... did I choose the wrong career?"*

## The Reality

AI isn't coming. **It's already here.**

The question isn't whether to use it.

The question is: **who will direct it?**

## You Have the Advantage

You understand:
- How software works
- What good code looks like
- How to break down problems
- When something is wrong

**You can direct these agents.** Most people can't.

## The Opportunity

Those who learn to control and direct agents will:
- Build faster
- Prototype ideas in hours, not weeks
- Bring more ideas to life
- Multiply their impact

**This is a superpower. Learn to use it.**